In [1]:
# Import necessary libraries and packages
import math
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from qiskit import Aer, IBMQ, QuantumCircuit
from qiskit.primitives import Estimator
from qiskit.providers.aer import StatevectorSimulator
from qiskit.utils import QuantumInstance

from qiskit.tools.jupyter import *
from qiskit.visualization import *

# Import Qiskit libraries for VQE
from qiskit.algorithms.optimizers import SLSQP, SPSA, COBYLA

# Import Qiskit Nature libraries
from qiskit_nature.algorithms import GroundStateEigensolver, VQEUCCFactory
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.circuit.library import UCC, UCCSD
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import BravyiKitaevMapper, JordanWignerMapper, ParityMapper
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer


from qiskit.primitives import Estimator
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock


In [3]:
# define ansatz , optimizer, estimator
from qiskit.primitives import Estimator, BackendEstimator
#from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms import VQE
from qiskit_nature.units import DistanceUnit

In [4]:
hydrogen_t = [["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]]
                  
h3p = Molecule( 
    geometry= hydrogen_t,
   charge=0, multiplicity=1, 
)

driver = ElectronicStructureMoleculeDriver(h3p, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF) 

problem = driver.run()

In [13]:
# Check the occupation of the spin orbitals
PN_property = problem.get_property("ParticleNumber")

# Define the active space around the Fermi level 
transformer = ActiveSpaceTransformer(
    num_electrons=2, # Number of electrons in our active space
    num_molecular_orbitals=2, # Number of orbitals in our active space
)

# Now you can get the reduced electronic structure problem
problem_reduced = ElectronicStructureProblem(driver, transformers=[transformer]) 

# The second quantized Hamiltonian of the reduce problem
second_q_ops_reduced = problem_reduced.second_q_ops()

In [14]:
num_alpha_electrons = problem.get_property('ParticleNumber').num_alpha
num_beta_electrons = problem.get_property('ParticleNumber').num_beta
num_spin_orbitals = int(problem.get_property('ParticleNumber').num_spin_orbitals)

nuclear_rep_energy = problem.get_property('ElectronicEnergy').nuclear_repulsion_energy
print("number of alpha electrons: " , num_alpha_electrons)
print("number of beta electrons: " , num_beta_electrons)
print("number of spin orbitals: " , num_spin_orbitals)
print("nuclear repulsion energy: " , nuclear_rep_energy)

number of alpha electrons:  1
number of beta electrons:  1
number of spin orbitals:  4
nuclear repulsion energy:  0.7199689944489797


In [15]:
# Setup the mapper and qubit converter
mapper_type = 'ParityMapper'

if mapper_type == 'ParityMapper':
    mapper = ParityMapper()
elif mapper_type == 'JordanWignerMapper':
    mapper = JordanWignerMapper()
elif mapper_type == 'BravyiKitaevMapper':
    mapper = BravyiKitaevMapper()


converter = QubitConverter(mapper)

qubit_op = converter.convert(second_q_ops_reduced["ElectronicEnergy"])
print(qubit_op)

-0.8105479805373279 * IIII
+ 0.1721839326191554 * IIIZ
- 0.22575349222402372 * IIZZ
+ 0.17218393261915543 * IZZI
- 0.2257534922240237 * ZZII
+ 0.12091263261776627 * IIZI
+ 0.16892753870087907 * IZZZ
+ 0.045232799946057826 * ZXIX
- 0.045232799946057826 * IXZX
- 0.045232799946057826 * ZXZX
+ 0.045232799946057826 * IXIX
+ 0.1661454325638241 * ZZIZ
+ 0.1661454325638241 * IZIZ
+ 0.17464343068300453 * ZZZZ
+ 0.12091263261776627 * ZIZI


In [16]:
# Set the mapper to qubits
parity_mapper = ParityMapper() # This is the example of parity mapping

# Set the qubit converter with two qubit reduction to reduce the computational cost 
parity_converter = QubitConverter(parity_mapper, two_qubit_reduction=True)    

# Compute the Hamitonian in qubit form
qubit_op_parity = parity_converter.convert(second_q_ops_reduced.get('ElectronicEnergy'), num_particles=problem_reduced.num_particles)

print(qubit_op_parity)

-1.05237324577286 * II
+ 0.39793742484317896 * IZ
- 0.39793742484317896 * ZI
- 0.01128010425623538 * ZZ
+ 0.18093119978423122 * XX


In [17]:
vqe_factory = VQEUCCFactory( # This is an example of UCC"SD" ansatz
    quantum_instance=Aer.get_backend("aer_simulator_statevector"),
    optimizer=SLSQP(),
    ansatz=UCC(excitations='sd')
) 

from qiskit.algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

solver = GroundStateEigensolver(parity_converter, vqe_factory)  
real_solution_t = solver.solve(problem_reduced).total_energies[0]    
print('Reference energy : ', real_solution_t)

Reference energy :  (-1.137306035695619+0j)


In [18]:
# Define our 'ansatz' for the problem
ansatz = UCC(
    qubit_converter=parity_converter,
    num_particles=problem_reduced.num_particles, 
    num_spin_orbitals=problem_reduced.num_spin_orbitals,
    excitations='sd'
)

In [19]:
from qiskit.primitives import Estimator, BackendEstimator

In [22]:
%%time
from qiskit.utils import algorithm_globals
algorithm_globals.random_seed = 1024

# Define convergence list
convergence = []

# Keep track of jobs 
job_list = []

# Initialize estimator object
estimator =Estimator() 

# Define evaluate_expectation function
def evaluate_expectation(x):
    x = list(x)
    
    # Define estimator run parameters
    job = estimator.run(circuits=ansatz,observables=qubit_op_parity, parameter_values=[x]).result()
    results = job.values[0]
    job_list.append(job)
    
    # Pass results back to callback function
    return np.real(results)

# Call back function
def callback(x,fx,ax,tx,nx):
    # Callback function to get a view on internal states and statistics of the optimizer for visualization
    convergence.append(evaluate_expectation(fx))



CPU times: user 124 µs, sys: 0 ns, total: 124 µs
Wall time: 143 µs


In [23]:
np.random.seed(10)

# Define initial point. We shall define a random point here based on the number of parameters in our ansatz
initial_point = np.random.random(ansatz.num_parameters)

#### enter your code below ####
# Define optimizer and pass callback function
optimizer = SPSA(maxiter=50,callback=callback)

# Define minimize function
result =optimizer.minimize(evaluate_expectation ,x0=initial_point)

In [29]:
from qiskit.algorithms import MinimumEigensolverResult, VQE

Energy_H_t = []
for i in range(len(convergence)):
    sol = MinimumEigensolverResult()
    sol.eigenvalue = convergence[i]
    sol = problem_reduced.interpret(sol).total_energies[0]
    Energy_H_t.append(sol)
print("Computed Energy:", Energy_H_t[-1])

Computed Energy: (-1.1369975436673183+0j)


## VQE for Bond_Distances

In [65]:
def get_qubit_op(dist):
    # Define Molecule
    molecule = Molecule(
        # Coordinates in Angstrom
        geometry=[
            ["H", [0.0, 0.0, dist]],
            ["H", [0.0, 0.0, 0.0]]
        ],
        multiplicity=1,  
        charge=0,
    )
    
    driver = ElectronicStructureMoleculeDriver(molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF) 

    # Run the preliminary quantum chemistry calculation
    properties = driver.run()

    # Set the active space
    active_space_trafo = ActiveSpaceTransformer(num_electrons=2,
                                        num_molecular_orbitals=2) 

    # Now you can get the reduced electronic structure problem
    problem_reduced = ElectronicStructureProblem(driver, transformers=[transformer]) 

    # The second quantized Hamiltonian of the reduce problem
    second_q_ops_reduced = problem_reduced.second_q_ops()

    # Set the mapper to qubits
    parity_mapper = ParityMapper() # This is the example of parity mapping

    # Set the qubit converter with two qubit reduction to reduce the computational cost 
    parity_converter = QubitConverter(parity_mapper, two_qubit_reduction=True)    

    # Compute the Hamitonian in qubit form
    qubit_op_parity = parity_converter.convert(second_q_ops_reduced.get('ElectronicEnergy'), num_particles=problem_reduced.num_particles)
   
    # Get reference solution
    vqe_factory = VQEUCCFactory(quantum_instance=StatevectorSimulator(),optimizer=SLSQP(),ansatz=UCC(excitations='sd')) 
    solver = GroundStateEigensolver(parity_converter, vqe_factory)    
    real_solution = solver.solve(problem_reduced).total_energies[0]    
    
    ansatz=vqe_factory.ansatz
    
    return ansatz, qubit_op_parity, real_solution, problem_reduced
    

In [66]:
def custom_vqe(estimator, ansatz, ops, problem_reduced):

    # Define convergence list
    convergence = []

    # Keep track of jobs (Do-not-modify)
    job_list = []

    # Define evaluate_expectation function
    # Define convergence list
    convergence = []

    # Keep track of jobs 
    job_list = []

    # Initialize estimator object
    estimator =Estimator() 

    # Define evaluate_expectation function
    def evaluate_expectation(x):
        x = list(x)

        # Define estimator run parameters
        job = estimator.run(circuits=ansatz,observables=qubit_op_parity, parameter_values=[x]).result()
        results = job.values[0]
        job_list.append(job)
    
    # Pass results back to callback function
        return np.real(results)

    # Call back function
    def callback(x,fx,ax,tx,nx):
        # Callback function to get a view on internal states and statistics of the optimizer for visualization
        convergence.append(evaluate_expectation(fx))

    

    # Define initial point. We shall define a random point here based on the number of parameters in our ansatz
    np.random.seed(10)

    # Define initial point. We shall define a random point here based on the number of parameters in our ansatz
    initial_point = np.random.random(ansatz.num_parameters)

    #### enter your code below ####
    # Define optimizer and pass callback function
    optimizer = SPSA(maxiter=50,callback=callback)

    # Define minimize function
    result =optimizer.minimize(evaluate_expectation ,x0=initial_point)
    
    vqe_interpret = []
    for i in range(len(convergence)):
        sol = MinimumEigensolverResult()
        sol.eigenvalue = convergence[i]
        sol = problem_reduced.interpret(sol).total_energies[0]
        vqe_interpret.append(sol)

    return vqe_interpret

In [67]:
distances = np.arange(0.5, 3.0, 0.2)

for dist in distances:

    ansatz, qubit_op_parity, real_solution, problem_reduced= get_qubit_op(dist)

    # Estimator VQE for H2
    Energy_H = custom_vqe(estimator=Estimator(), ansatz=ansatz, ops= qubit_op_parity, problem_reduced=problem_reduced)

print(Energy_H)

[(-0.6088254197798731+0j), (-0.4717065830143232+0j), (-0.47170788383334994+0j), (-0.4717091342498979+0j), (-0.6282723815242559+0j), (-0.6419480314986352+0j), (-0.8131533595957696+0j), (-0.8363754187377992+0j), (-0.9042125302401776+0j), (-0.9042532460315416+0j), (-0.9242502372118455+0j), (-0.9248373071957927+0j), (-0.924858801958099+0j), (-0.9305841544630646+0j), (-0.9316601217473022+0j), (-0.931694747977658+0j), (-0.9324683573203871+0j), (-0.9335106864962512+0j), (-0.9335107902108877+0j), (-0.9334868249387334+0j), (-0.9334639925957844+0j), (-0.9334453878026923+0j), (-0.9334479713127859+0j), (-0.9334411862796888+0j), (-0.9334435540325876+0j), (-0.933441567972831+0j), (-0.9334411790736368+0j), (-0.9335406568880293+0j), (-0.9335303842777456+0j), (-0.9335169741450917+0j), (-0.9335100314169023+0j), (-0.9335307620767015+0j), (-0.9335185288870056+0j), (-0.9335364688904726+0j), (-0.9335241042299838+0j), (-0.9335141174760846+0j), (-0.9335435326105104+0j), (-0.9335439727130213+0j), (-0.933532730